In [1]:
import gate
import numpy as np
%load_ext autoreload
%autoreload 2
init = [1, 0, 0, 0]
print(gate.gate2['CX'] @ (np.kron(gate.gate1['H'],gate.gate1['I']) @ init))

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [10]:
for i in range(0, 10):
    k = np.kron(gate.gate1['I'],gate.gate1['H'])


In [ ]:
for i in range(0, 1000000):
    k = gate.gate2['IH']

In [27]:
def kron_A_I(A, N):
    out = np.zeros((N*N, N*N),dtype=A.dtype)
    for i in range(N):
        for j in range(N):
            value = A[i,j]
            out[i*2: i*2 + 2,j*2:j*2 + 2] = np.array([[value, 0], [0, value]])
    return out

In [68]:
A = np.random.rand(4, 4)
print(A)

[[0.08438087 0.41107181 0.04538587 0.95188399]
 [0.1705832  0.77136947 0.69109793 0.58784028]
 [0.29480806 0.77784422 0.59064393 0.70858758]
 [0.11067381 0.13596001 0.74064147 0.29661303]]


In [3]:
import numpy as np

def kron_A_X(A):
    N = A.shape[0]
    out = np.zeros((N*2, N*2),dtype=A.dtype)
    for i in range(N):
        for j in range(N):
            value = A[i,j]
            out[i*2: i*2 + 2,j*2:j*2 + 2] = np.array([[0, value], [value, 0]])
    return out


A = np.random.rand(3, 3)
result = kron_A_X(A)
print(result)


[[0.         0.34270731 0.         0.84177329 0.         0.58071937]
 [0.34270731 0.         0.84177329 0.         0.58071937 0.        ]
 [0.         0.89888455 0.         0.51334885 0.         0.28378635]
 [0.89888455 0.         0.51334885 0.         0.28378635 0.        ]
 [0.         0.64201619 0.         0.64069361 0.         0.25695429]
 [0.64201619 0.         0.64069361 0.         0.25695429 0.        ]]


In [78]:
np.allclose(np.kron(A, np.array([[0, 1], [1, 0]])), result)

True

In [2]:
A = np.random.rand(300, 300)
A = A.astype(np.complex128)
B = np.array([[1,1],[1,1]])

In [32]:
for i in range(1000):
    k = np.kron(A, gate.gate2['CRZ'](np.pi/4))

In [33]:
np.allclose(k1, k)

True

In [31]:
for i in range(1000):
    k1 = ACRZ(A, np.pi/4)

In [30]:
def ACRX(A, theta):
    cos_theta_2_times_A = np.cos(theta / 2) * A
    sin_theta_2_times_A = -1j * np.sin(theta / 2) * A
    m, n = A.shape
    out = np.zeros((m, 4, n, 4), dtype=complex)
    out[:, 0, :, 0] = A
    out[:, 1, :, 1] = A
    out[:, 2, :, 2] = cos_theta_2_times_A
    out[:, 2, :, 3] = sin_theta_2_times_A
    out[:, 3, :, 2] = sin_theta_2_times_A
    out[:, 3, :, 3] = cos_theta_2_times_A
    out.shape = (4 * m, 4 * n)
    return out

def CRXA(A, theta):
    cos_theta_2_times_A = np.cos(theta / 2) * A
    sin_theta_2_times_A = -1j * np.sin(theta / 2) * A
    m, n = A.shape
    out = np.zeros((4, m, 4, n), dtype=complex)
    out[0, :, 0, :] = A
    out[1, :, 1, :] = A
    out[2, :, 2, :] = cos_theta_2_times_A
    out[2, :, 3, :] = sin_theta_2_times_A
    out[3, :, 2, :] = sin_theta_2_times_A
    out[3, :, 3, :] = cos_theta_2_times_A
    out.shape = (4 * m, 4 * n)
    return out

def ACRY(A, theta):
    cos_theta_2_times_A = np.cos(theta / 2) * A
    sin_theta_2_times_A = np.sin(theta / 2) * A

    m, n = A.shape
    out = np.zeros((m, 4, n, 4), dtype=complex)
    
    out[:, 0, :, 0] = A
    out[:, 1, :, 1] = A
    out[:, 2, :, 2] = cos_theta_2_times_A
    out[:, 2, :, 3] = -sin_theta_2_times_A
    out[:, 3, :, 2] = sin_theta_2_times_A
    out[:, 3, :, 3] = cos_theta_2_times_A
    
    out.shape = (4 * m, 4 * n)
    return out

def CRYA(A, theta):
    cos_theta_2_times_A = np.cos(theta / 2) * A
    sin_theta_2_times_A = np.sin(theta / 2) * A

    m, n = A.shape
    out = np.zeros((4, m, 4, n), dtype=complex)
    
    out[0, :, 0, :] = A
    out[1, :, 1, :] = A
    out[2, :, 2, :] = cos_theta_2_times_A
    out[2, :, 3, :] = -sin_theta_2_times_A
    out[3, :, 2, :] = sin_theta_2_times_A
    out[3, :, 3, :] = cos_theta_2_times_A
    
    out.shape = (4 * m, 4 * n)
    return out


def CRZA(A, theta):
    phase = 1j * theta / 2
    m, n = A.shape
    out = np.zeros((4, m, 4, n), dtype=complex)
    out[0, :, 0, :] = A
    out[1, :, 1, :] = A
    out[2, :, 2, :] = np.exp(-phase) * A
    out[3, :, 3, :] = np.exp(phase) * A
    out.shape = (4 * m, 4 * n)
    return out

def ACRZ(A, theta):
    phase = 1j * theta / 2
    m, n = A.shape
    out = np.zeros((m, 4, n, 4), dtype=complex)
    out[:, 0, :, 0] = A
    out[:, 1, :, 1] = A
    out[:, 2, :, 2] = np.exp(-phase) * A
    out[:, 3, :, 3] = np.exp(phase) * A
    out.shape = (4 * m, 4 * n)
    return out
